In [1]:
import numpy as np
import pandas as pd
import json
import runProcs
from urllib.request import urlopen
import os

import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline

In [2]:
# 0. Import BEA API key or set manually to variable api_key
try:
    items = os.getcwd().split('/')[:3]
    items.append('bea_api_key.txt')
    path = '/'.join(items)
    with open(path,'r') as api_key_file:
        api_key = api_key_file.readline()

except:
    api_key = None

In [3]:
# 1. State abbreviations

# 1.1 dictionary:
stateAbbr = {
u'Alabama':u'AL',
u'Alaska *':u'AK',
u'Arizona':u'AZ',
u'Arkansas':u'AR',
u'California':u'CA',
u'Colorado':u'CO',
u'Connecticut':u'CT',
u'Delaware':u'DE',
u'District of Columbia':u'DC',
u'Florida':u'FL',
u'Georgia':u'GA',
u'Hawaii *':u'HI',
u'Idaho':u'ID',
u'Illinois':u'IL',
u'Indiana':u'IN',
u'Iowa':u'IA',
u'Kansas':u'KS',
u'Kentucky':u'KY',
u'Louisiana':u'LA',
u'Maine':u'ME',
u'Maryland':u'MD',
u'Massachusetts':u'MA',
u'Michigan':u'MI',
u'Minnesota':u'MN',
u'Mississippi':u'MS',
u'Missouri':u'MO',
u'Montana':u'MT',
u'Nebraska':u'NE',
u'Nevada':u'NV',
u'New Hampshire':u'NH',
u'New Jersey':u'NJ',
u'New Mexico':u'NM',
u'New York':u'NY',
u'North Carolina':u'NC',
u'North Dakota':u'ND',
u'Ohio':u'OH',
u'Oklahoma':u'OK',
u'Oregon':u'OR',
u'Pennsylvania':u'PA',
u'Rhode Island':u'RI',
u'South Carolina':u'SC',
u'South Dakota':u'SD',
u'Tennessee':u'TN',
u'Texas':u'TX',
u'Utah':u'UT',
u'Vermont':u'VT',
u'Virginia':u'VA',
u'Washington':u'WA',
u'West Virginia':u'WV',
u'Wisconsin':u'WI',
u'Wyoming':u'WY'
}

# 1.2 List of states in the US
stateList = [s for s in stateAbbr]

In [4]:
# 2. Construct series for price deflator

# 2.1 Obtain data from BEA
gdp_deflator = urlopen('http://apps.bea.gov/api/data/?UserID='+api_key+'&method=GetData&datasetname=NIPA&TableName=T10109&TableID=13&Frequency=A&Year=X&ResultFormat=JSON&')
# result = gdp_deflator.readall().decode('utf-8')
result = gdp_deflator.read().decode('utf-8')
json_response = json.loads(result)

In [5]:
# 2.2 Construct the data frame for the deflator series
values = []
years = []
for element in json_response['BEAAPI']['Results']['Data']:
#     if element['LineDescription'] == 'Personal consumption expenditures':
    if element['LineDescription'] == 'Gross domestic product':
        years.append(element['TimePeriod'])
        values.append(float(element['DataValue'])/100)

values = np.array([values]).T
data_p = pd.DataFrame(values,index = years,columns = ['price level'])

# 2.3 Display the data
print(data_p)

      price level
1929      0.09424
1930      0.09079
1931      0.08146
1932      0.07193
1933      0.06993
...           ...
2016      1.05722
2017      1.07710
2018      1.10296
2019      1.12265
2020      1.13625

[92 rows x 1 columns]


In [6]:
# 3. Construct series for per capita income by state, region, and the entire us

# 3.1 Obtain data from BEA
state_y_pc = urlopen('http://apps.bea.gov/api/data/?UserID='+api_key+'&method=GetData&DataSetName=Regional&TableName=SAINC1&LineCode=3&Year=ALL&GeoFips=STATE&ResultFormat=JSON')
# result = state_y_pc.readall().decode('utf-8')
result = state_y_pc.read().decode('utf-8')
json_response = json.loads(result)
# json_response['BEAAPI']['Results']['Data'][0]['GeoName']

In [7]:
# 3.2 Construct the data frame for the per capita income series

# 3.2.1 Initialize the dataframe
regions = []
years = []
for element in json_response['BEAAPI']['Results']['Data']:
    if element['GeoName'] not in regions:
        regions.append(element['GeoName'])
    if element['TimePeriod'] not in years:
        years.append(element['TimePeriod'])

df = np.zeros([len(years),len(regions)])
data_y = pd.DataFrame(df,index = years,columns = regions)

# 3.2.2 Populate the dataframe with values
for element in json_response['BEAAPI']['Results']['Data']:
    try:
        data_y[element['GeoName']][element['TimePeriod']] = np.round(float(element[u'DataValue'].replace(',',''))/float(data_p.loc[element['TimePeriod']]),2)# real
    except:
        data_y[element['GeoName']][element['TimePeriod']] = np.nan
        
# 3.2.3 Replace the state names in the index with abbreviations
columns=[]
for r in regions:
    if r in stateList:
        columns.append(stateAbbr[r])
    else:
        columns.append(r)
        
data_y.columns=columns

# 3.2.4 Display the data obtained from the BEA
data_y

,United States,AL,AK,AZ,AR,CA,CO,CT,DE,DC,...,WI,WY,New England,Mideast,Great Lakes,Plains,Southeast,Southwest,Rocky Mountain,Far West
1929,7417.23,3384.97,NaN,6345.50,3215.20,10547.54,6695.67,10918.93,10940.15,14049.24,...,7109.51,7130.73,9295.42,10314.09,8446.52,6005.94,3851.87,4987.27,6250.00,9603.14
1930,6839.96,2896.79,NaN,5705.47,2467.23,9791.83,6322.28,10210.38,9450.38,14472.96,...,6443.44,6410.40,8877.63,9802.84,7467.78,5551.27,3414.47,4383.74,5859.68,8943.72
1931,6481.71,2700.71,NaN,5278.66,2541.12,9243.80,5794.25,9906.70,9526.15,15381.78,...,5757.43,5855.63,8814.14,9391.11,6935.92,5119.08,3326.79,4087.90,5155.90,8384.48
1932,5588.77,2210.48,NaN,4476.57,2113.17,8119.00,4963.16,8716.81,8216.32,15389.96,...,5032.67,5213.40,7979.98,8244.13,5699.99,4351.45,2822.19,3475.60,4587.79,7312.67
1933,5362.51,2345.20,NaN,4418.70,2187.90,7865.01,5047.91,8451.31,8093.81,13513.51,...,4776.20,5319.61,7650.51,7793.51,5405.41,3975.40,2945.80,3517.80,4490.20,7078.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016,47289.12,37370.65,53174.36,38571.92,38175.59,54930.86,49593.27,66103.55,46140.82,73427.48,...,45001.99,51146.40,58882.73,55645.94,44575.40,45636.67,41433.19,41830.46,44933.88,53016.40
2017,48366.91,38065.18,53193.76,39519.08,38642.65,56244.55,51573.67,66566.71,47106.12,73320.95,...,45714.42,52382.32,59887.66,57229.60,45235.35,46023.58,42357.26,43197.47,46364.31,54312.51
2018,49485.93,38692.25,54611.23,40398.56,39248.93,57807.17,53343.73,67809.35,47914.70,73387.07,...,46793.17,54885.94,61162.69,58290.42,46200.22,47009.86,43224.60,44459.45,47949.16,55876.01
2019,50304.19,39283.84,55786.75,40952.21,39711.40,59453.08,54477.35,68830.89,48388.19,74031.09,...,47394.11,55265.67,62114.64,59513.65,46730.50,47641.74,43738.48,45122.70,48870.97,57284.10


In [8]:
# 4. State income data for 1840, 1880, and 1900

# 4.1.1 Import Easterlin's income data
easterlin_data = pd.read_csv('../historic_data/Historical Statistics of the US - Easterlin State Income Data.csv',index_col=0)

# 4.1.2 Import historic CPI data
historic_cpi_data=pd.read_csv('../historic_data/Historical Statistics of the US - cpi.csv',index_col=0)
historic_cpi_data = historic_cpi_data/historic_cpi_data.loc[1929]*float(data_p.loc['1929'])

In [9]:
# 4.2 Append to data beginning in 1929

# 4.2.1 Construct series for real incomes in 1840, 1880, and 1900
df_1840 = easterlin_data['Income per capita - 1840 - A [cur dollars]']/float(historic_cpi_data.loc[1840])
df_1880 = easterlin_data['Income per capita - 1880 [cur dollars]']/float(historic_cpi_data.loc[1890])
df_1900 = easterlin_data['Income per capita - 1900 [cur dollars]']/float(historic_cpi_data.loc[1900])

# 4.2.2 Put into a DataFrame and concatenate with previous
df = pd.DataFrame({'1840':df_1840,'1880':df_1880,'1900':df_1900}).transpose()
df = pd.concat([data_y,df]).sort_index()

In [10]:
# 5. Export data to csv
series = df.sort_index()
dropCols = [u'AK', u'HI', u'New England', u'Mideast', u'Great Lakes', u'Plains', u'Southeast', u'Southwest', u'Rocky Mountain', u'Far West']
for c in dropCols:
    series = series.drop([c],axis=1)

series.to_csv('../csv/state_income_data.csv',na_rep='NaN')

In [11]:
# 6. Export notebook to .py
runProcs.exportNb('state_income_data')